In [1]:
!python setup.py build_ext --inplace

running build_ext
running build_ext


In [2]:
import network_simulation_freeze_stop as net_sims
import popconstruct as pops
import plotting_helper_functions as plt_help
import pandas as pd
import os
import time
import pdb

import pylab as pl
import pickle
import numpy as np
import pandas as pd
import pingouin
import sklearn 

def saveresults_vars(variable, prefix):
    pickle.dump(variable, open(prefix, 'wb'))
    
def loadresults_vars(prefix):
    return pickle.load(open(prefix, "rb"))

data_dir = "./Data/stop-params_freeze-analysis/STN+iSPN+GPeA_10threads/04-all/"
figure_dir = "./Figures/"

2023-08-18 11:53:53,137	INFO worker.py:1352 -- Connecting to existing Ray cluster at address: 127.0.0.1:6379...
2023-08-18 11:53:53,145	INFO worker.py:1529 -- Connected to Ray cluster. View the dashboard at http://127.0.0.1:8265 


In [3]:
experimentchoice = 'stopsignal'
number_of_choices = 1
#pl_creat.choose_pipeline(experimentchoice)

In [4]:
environment = {
    
    'experimentchoice': experimentchoice,
    'inter_trial_interval': None,
    'thalamic_threshold': 30.,
    'movement_time': ['constant', 300], #['constant', 300], #default sampled from N(250,1.5), ["constant",250], ["mean",250]
    'choice_timeout': 300, #default 1000
    
    'params': None, #neuron parameters (init_params.py)
    'pops': None,   #population parameters (init_params.py)
    'receps' : None, #receptor parameters (init_params.py)
    'base' : None, #baseline stimulation parameters (init_params.py)
    'dpmns' : None, #dopamine related parameters (init_params.py)
    'd1' : None, #D1-MSNs population related parameters (init_params.py)
    'd2' : None, #D2-MSNs population related parameters (init_params.py)
    'channels' : pd.DataFrame([["left"]], columns=['action']), #action channels related parameters (init_params.py)
    'number_of_choices': number_of_choices,
    #'actionchannels' : pd.DataFrame([[1],[2]], columns=['action']), #labels for the actions (init_params.py)
    #'actionchannels' : pd.DataFrame([["left"],["right"]], columns=['action']), #labels for the actions (init_params.py)
    'newpathways' : None, #connectivity parameters (popconstruct.py)
    'Q_support_params': None, #initialization of Q-values update (qvalues.py) 
    'Q_df_set': pd.DataFrame([[0.5]],columns=["left"]), #pd.DataFrame([[0.3,0.7]],columns=['left','right']), #initialized Q-values df (qvalues.py)  
    'n_trials': 30, #number of trials (generateepochs.py)
    'volatility': [None,"exact"], #frequency of changepoints (generateepochs.py)
    'conflict': (1.0), #probability of the preferred choice (generateepochs.py)
    'reward_mu': 1, #mean for the magnitude of the reward (generateepochs.py)
    'reward_std': 0.1, #std for the magnitude of the reward (generateepochs.py)
    'maxstim': 1.05, # amplitude of the cortical input over base line #0.85
    'sustainedfraction': 0.75,
    
    #Stop 1
    'stop_signal_present': True,
    'stop_signal_probability': 1., #probability of trials that will get the stop signal / list of trial numbers
    'stop_signal_amplitude': 0.4, # amplitude of the stop signal over base line
    'stop_signal_onset': 30., #in ms #antes 200
    'stop_signal_duration' : 150., 
    'stop_signal_channel': "all", #"all" (all channels are given the stop signal) 
                                #/"any" (channel given the stop signal is chosen randomly)
                                # [list of channels] == subset of channels given the stop signa
    'stop_signal_population':["STNE"],
    
    #Stop 2
    'stop_2_signal_present': True,
    'stop_2_signal_probability': 1., #probability of trials that will get the stop signal / list of trial numbers
    'stop_2_signal_amplitude': 0.4, # amplitude of the stop signal over base line
    'stop_2_signal_onset': 60., #in ms #antes 200
    'stop_2_signal_duration' : 150.,
    'stop_2_signal_channel': "all", #"all" (all channels are given the stop signal) 
                                #/"any" (channel given the stop signal is chosen randomly)
                                # [list of channels] == subset of channels given the stop signal
    'stop_2_signal_population':["GPeA"],                           
    
    
    'decision_channel' : 'all', #'all'(all channels)
                              # '0' : left channel considered 
                              # '1' : right channel considered
    
    'record_variables': [], #["weight","optogenetic_input", "stop_input_1", "stop_input_2"],
    
    #Opto
    'opt_signal_present': True,
    'opt_signal_probability': 1., #[0,1,2,3,4], # probability of trials that will get the optogenetic signal / list of trial numbers
    'opt_signal_amplitude': 0.4, # ampitude of the stop signal over base line
    'opt_signal_onset': 60., # in ms
    'opt_signal_duration': 120.,
    'opt_signal_channel': "all", # "all" (all channels are given the stop signal) 
    'opt_signal_population':["D2STR"],    
}

In [5]:
solutions_onset = np.arange(70, 140, 10) #np.arange(0.52, 0.64, 0.1) #np.arange(0.42, 0.63, 0.1)
solutions_duration =  145. #np.arange(80, 210, 20) #np.arange(0.02, 0.63, 0.1)

print(solutions_onset)
print(solutions_duration)

[  0  10  20  30  40  50  60  70  80  90 100]
[ 80 100 120 140 160 180 200]


In [6]:
offset = 0

In [7]:
for i in np.arange(len(solutions_onset)):
    if i < offset:
        continue
    print("i:",i)
                 
    onset = solutions_onset[i]
    
    for j in np.arange(len(solutions_duration)):
        
        print("j:", j) 
        
        seed_1 = i
        seed_2 = j
        
        duration = solutions_duration[j]
        
        if os.path.exists(data_dir+'network_data_onset_'+str(onset)+'_duration_'+str(duration)):
            continue
        
        new_env = environment.copy()
        new_env['stop_signal_onset'] = onset
        new_env['stop_2_signal_onset'] = onset
        new_env['opt_signal_onset'] = onset
        
        new_env['stop_signal_duration'] = duration
        new_env['stop_2_signal_duration'] = duration
        new_env['opt_signal_duration'] = duration
        
        print('Onset:', onset)
        print('Duration:', duration)

        results = net_sims.run_simulation(seed_1, seed_2, onset, duration, new_env)

i: 0
j: 0
j: 1
j: 2
j: 3
j: 4
j: 5
j: 6
i: 1
j: 0
j: 1
j: 2
j: 3
j: 4
j: 5
j: 6
i: 2
j: 0
j: 1
j: 2
j: 3
j: 4
j: 5
j: 6
i: 3
j: 0
j: 1
j: 2
j: 3
j: 4
j: 5
j: 6
i: 4
j: 0
j: 1
j: 2
j: 3
j: 4
j: 5
j: 6
i: 5
j: 0
j: 1
j: 2
j: 3
j: 4
j: 5
j: 6
i: 6
j: 0
j: 1
j: 2
j: 3
j: 4
j: 5
j: 6
i: 7
j: 0
j: 1
j: 2
j: 3
j: 4
j: 5
j: 6
i: 8
j: 0
j: 1
j: 2
j: 3
j: 4
j: 5
j: 6
i: 9
j: 0
j: 1
j: 2
j: 3
j: 4
j: 5
j: 6
i: 10
j: 0
j: 1
j: 2
j: 3
j: 4
j: 5
j: 6
Onset: 100
Duration: 200
{'experimentchoice': 'stopsignal', 'inter_trial_interval': None, 'thalamic_threshold': 30.0, 'movement_time': ['constant', 300], 'choice_timeout': 300, 'params': None, 'pops': None, 'receps': None, 'base': None, 'dpmns': None, 'd1': None, 'd2': None, 'channels':   action
0   left, 'number_of_choices': 1, 'newpathways': None, 'Q_support_params': None, 'Q_df_set':    left
0   0.5, 'n_trials': 30, 'volatility': [None, 'exact'], 'conflict': 1.0, 'reward_mu': 1, 'reward_std': 0.1, 'maxstim': 1.05, 'sustainedfraction': 0.75, 'stop_sig

(worker pid=1503) channels: one
(worker pid=1503) scaling_conn 1
(worker pid=1503) scaling_wts 2.0
(worker pid=1503) channels: one
(worker pid=1503) scaling_conn 1
(worker pid=1503) scaling_wts 2.0
SpawnThreadManagers <backend.Pipeline object at 0x7fc0135aeeb0>
SpawnThreadManagers <backend.Pipeline object at 0x7fc0135b7d30>
SpawnThreadManagers <backend.Pipeline object at 0x7fc0135aeeb0>
(worker pid=1503) channels: one
(worker pid=1503) scaling_conn 1
(worker pid=1503) scaling_wts 2.0
(worker pid=1535) [11 13 19 15  1  2  7 23 20 24 28 10 18 27 16 12  5  4 21  8  0 29 25 26
(worker pid=1535)   9 14 22  3 17  6]
(worker pid=1503) channels: one
(worker pid=1503) scaling_conn 1
(worker pid=1503) scaling_wts 2.0
(worker pid=1535) [11 13 19 15  1  2  7 23 20 24 28 10 18 27 16 12  5  4 21  8  0 29 25 26
(worker pid=1535)   9 14 22  3 17  6]
(worker pid=1535) [ 2 28 13 10 26 24 27 11 17 22  5 16  8 14 23 20  1 29  6  4 18 19  9  7
(worker pid=1535)  25  3  0 21 15 12]
(worker pid=1535) [ 2 28 

SpawnThreadManagers <backend.Pipeline object at 0x7fc0135b4910>
SpawnThreadManagers <backend.Pipeline object at 0x7fc0135b4910>
(worker pid=1514) channels: one
(worker pid=1514) scaling_conn 1
(worker pid=1514) scaling_wts 2.0
(worker pid=1514) channels: one
(worker pid=1514) scaling_conn 1
(worker pid=1514) scaling_wts 2.0
(worker pid=1539) channels: one
(worker pid=1539) scaling_conn 1
(worker pid=1539) scaling_wts 2.0
(worker pid=1539) channels: one
(worker pid=1539) scaling_conn 1
(worker pid=1539) scaling_wts 2.0
SpawnThreadManagers <backend.Pipeline object at 0x7fc0135b4910>
SpawnThreadManagers <backend.Pipeline object at 0x7fc0135b4910>
(worker pid=1514) channels: one
(worker pid=1514) scaling_conn 1
(worker pid=1514) scaling_wts 2.0
(worker pid=1514) channels: one
(worker pid=1514) scaling_conn 1
(worker pid=1514) scaling_wts 2.0
SpawnThreadManagers <backend.Pipeline object at 0x7fc0135b4910>
(worker pid=1514) channels: one
(worker pid=1514) scaling_conn 1
(worker pid=1514) sca

(worker pid=1505) stop stim started
(worker pid=1505) stop_2 stim started
(worker pid=1505) opt stim started
(worker pid=1505) stop stim started
(worker pid=1505) stop_2 stim started
(worker pid=1505) opt stim started
(worker pid=1505) gateFRs [30.22222222]
(worker pid=1505) thresholds_crossed [0]
(worker pid=1505) gateFRs [30.22222222]
(worker pid=1505) thresholds_crossed [0]
(worker pid=1537) stop stim started
(worker pid=1537) stop_2 stim started
(worker pid=1537) opt stim started
(worker pid=1537) stop stim started
(worker pid=1537) stop_2 stim started
(worker pid=1537) opt stim started
(worker pid=1542) stop stim started
(worker pid=1542) stop_2 stim started
(worker pid=1542) opt stim started
(worker pid=1542) stop stim started
(worker pid=1542) stop_2 stim started
(worker pid=1542) opt stim started
(worker pid=1539) stop stim started
(worker pid=1539) stop_2 stim started
(worker pid=1539) opt stim started
(worker pid=1539) stop stim started
(worker pid=1539) stop_2 stim started
(

(worker pid=1537) gateFRs [4.44444444]
(worker pid=1537) thresholds_crossed []
(worker pid=1503) trial_num 2
(worker pid=1503)   action
(worker pid=1503) 0   left
(worker pid=1503) chosen_action: left
(worker pid=1537) gateFRs [4.44444444]
(worker pid=1537) thresholds_crossed []
(worker pid=1503) trial_num 2
(worker pid=1503)   action
(worker pid=1503) 0   left
(worker pid=1503) chosen_action: left
(worker pid=1505) trial_num 2
(worker pid=1505)   action
(worker pid=1505) 0   left
(worker pid=1505) chosen_action: left
(worker pid=1542) gateFRs [4.44444444]
(worker pid=1542) thresholds_crossed []
(worker pid=1505) trial_num 2
(worker pid=1505)   action
(worker pid=1505) 0   left
(worker pid=1505) chosen_action: left
(worker pid=1542) gateFRs [4.44444444]
(worker pid=1542) thresholds_crossed []
(worker pid=1540) gateFRs [4.]
(worker pid=1540) thresholds_crossed []
(worker pid=1540) gateFRs [4.]
(worker pid=1540) thresholds_crossed []
(worker pid=1535) trial_num 2
(worker pid=1535)   acti

...

(worker pid=1503) trial_num 13
(worker pid=1503)   action
(worker pid=1503) 0   left
(worker pid=1503) chosen_action: stop
(worker pid=1503) trial_num 13
(worker pid=1503)   action
(worker pid=1503) 0   left
(worker pid=1503) chosen_action: stop
(worker pid=1535) stop stim started
(worker pid=1535) stop_2 stim started
(worker pid=1535) opt stim started
(worker pid=1535) stop stim started
(worker pid=1535) stop_2 stim started
(worker pid=1535) opt stim started
(worker pid=1542) gateFRs [30.22222222]
(worker pid=1542) thresholds_crossed [0]
(worker pid=1535) gateFRs [30.22222222]
(worker pid=1535) thresholds_crossed [0]
(worker pid=1542) gateFRs [30.22222222]
(worker pid=1542) thresholds_crossed [0]
(worker pid=1535) gateFRs [30.22222222]
(worker pid=1535) thresholds_crossed [0]
(worker pid=1542) stop stim started
(worker pid=1542) stop_2 stim started
(worker pid=1542) opt stim started
(worker pid=1542) stop stim started
(worker pid=1542) stop_2 stim started
(worker pid=1542) opt stim st

(worker pid=1535) stop stim started
(worker pid=1535) stop_2 stim started
(worker pid=1535) opt stim started
(worker pid=1535) stop stim started
(worker pid=1535) stop_2 stim started
(worker pid=1535) opt stim started
(worker pid=1535) gateFRs [6.22222222]
(worker pid=1535) thresholds_crossed []
(worker pid=1535) gateFRs [6.22222222]
(worker pid=1535) thresholds_crossed []
(worker pid=1542) trial_num 17
(worker pid=1542)   action
(worker pid=1542) 0   left
(worker pid=1542) chosen_action: left
(worker pid=1542) trial_num 17
(worker pid=1542)   action
(worker pid=1542) 0   left
(worker pid=1542) chosen_action: left
(worker pid=1503) stop stim started
(worker pid=1503) stop_2 stim started
(worker pid=1503) opt stim started
(worker pid=1503) stop stim started
(worker pid=1503) stop_2 stim started
(worker pid=1503) opt stim started
(worker pid=1535) trial_num 17
(worker pid=1535)   action
(worker pid=1535) 0   left
(worker pid=1535) chosen_action: stop
(worker pid=1535) trial_num 17
(worke

(worker pid=1535) trial_num 20
(worker pid=1535)   action
(worker pid=1535) 0   left
(worker pid=1535) chosen_action: stop
(worker pid=1535) trial_num 20
(worker pid=1535)   action
(worker pid=1535) 0   left
(worker pid=1535) chosen_action: stop
(worker pid=1542) trial_num 20
(worker pid=1542)   action
(worker pid=1542) 0   left
(worker pid=1542) chosen_action: stop
(worker pid=1542) trial_num 20
(worker pid=1542)   action
(worker pid=1542) 0   left
(worker pid=1542) chosen_action: stop
(worker pid=1503) trial_num 20
(worker pid=1503)   action
(worker pid=1503) 0   left
(worker pid=1503) chosen_action: left
(worker pid=1503) trial_num 20
(worker pid=1503)   action
(worker pid=1503) 0   left
(worker pid=1503) chosen_action: left
(worker pid=1535) gateFRs [30.44444444]
(worker pid=1535) thresholds_crossed [0]
(worker pid=1535) gateFRs [30.44444444]
(worker pid=1535) thresholds_crossed [0]
(worker pid=1535) stop stim started
(worker pid=1535) stop_2 stim started
(worker pid=1535) opt stim

(worker pid=1542) trial_num 23
(worker pid=1542)   action
(worker pid=1542) 0   left
(worker pid=1542) chosen_action: stop
(worker pid=1542) trial_num 23
(worker pid=1542)   action
(worker pid=1542) 0   left
(worker pid=1542) chosen_action: stop
(worker pid=1503) trial_num 23
(worker pid=1503)   action
(worker pid=1503) 0   left
(worker pid=1503) chosen_action: left
(worker pid=1503) trial_num 23
(worker pid=1503)   action
(worker pid=1503) 0   left
(worker pid=1503) chosen_action: left
(worker pid=1535) stop stim started
(worker pid=1535) stop_2 stim started
(worker pid=1535) opt stim started
(worker pid=1535) stop stim started
(worker pid=1535) stop_2 stim started
(worker pid=1535) opt stim started
(worker pid=1535) gateFRs [4.44444444]
(worker pid=1535) thresholds_crossed []
(worker pid=1535) gateFRs [4.44444444]
(worker pid=1535) thresholds_crossed []
(worker pid=1542) gateFRs [30.22222222]
(worker pid=1542) thresholds_crossed [0]
(worker pid=1542) gateFRs [30.22222222]
(worker pid

(worker pid=1535) stop stim started
(worker pid=1535) stop_2 stim started
(worker pid=1535) opt stim started
(worker pid=1535) stop stim started
(worker pid=1535) stop_2 stim started
(worker pid=1535) opt stim started
(worker pid=1535) gateFRs [4.66666667]
(worker pid=1535) thresholds_crossed []
(worker pid=1535) gateFRs [4.66666667]
(worker pid=1535) thresholds_crossed []
(worker pid=1542) gateFRs [30.22222222]
(worker pid=1542) thresholds_crossed [0]
(worker pid=1542) stop stim started
(worker pid=1542) stop_2 stim started
(worker pid=1542) opt stim started
(worker pid=1542) gateFRs [30.22222222]
(worker pid=1542) thresholds_crossed [0]
(worker pid=1542) stop stim started
(worker pid=1542) stop_2 stim started
(worker pid=1542) opt stim started
(worker pid=1535) trial_num 27
(worker pid=1535)   action
(worker pid=1535) 0   left
(worker pid=1535) chosen_action: stop
(worker pid=1535) trial_num 27
(worker pid=1535)   action
(worker pid=1535) 0   left
(worker pid=1535) chosen_action: sto